In [1]:
!pip install ultralytics --no-deps
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [2]:

from roboflow import Roboflow
rf = Roboflow(api_key="y3KXuDq5MlOgJvuPrnjN")
project = rf.workspace("shukur-sabzaliev1").project("volleyball_v2")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Volleyball_v2-1 in yolov8:: 100%|██████████| 37972/37972 [00:05<00:00, 6342.46it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from ultralytics import YOLO

# --- 1. SET YOUR PATHS ---
DATASET_YAML_PATH = '/kaggle/working/Volleyball_v2-1/data.yaml'

if __name__ == "__main__":
    # --- 2. LOAD MODEL ---
    model = YOLO('yolov8m.pt')  # include .pt for clarity

    # --- 3. TRAIN MODEL ---
    print("Starting YOLOv8m training...")

    results = model.train(
        data=DATASET_YAML_PATH,
        epochs=100,             # start with 100
        imgsz=640,
        project='runs/detect',
        name='volleyball_v2_finetune_all_aug',
        batch=64,               # safer default; increase if memory allows
        workers=8,
        half=True,              # FP16 training (useful on supported GPUs)
        device="0,1",           

        # --- AUGMENTATIONS ---
        mosaic=1.0,
        mixup=0.1,              # optional: small mixup helps robustness
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        fliplr=0.5,
        degrees=15,
        scale=0.1,
        shear=5,
        perspective=0.001,
        patience=20             # early stopping if val loss doesn't improve
    )

    print("✅ Training complete!")


Starting YOLOv8m training...
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/Volleyball_v2-1/data.yaml, degrees=15, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=volleyball_v2_f

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1898       2241      0.914      0.772      0.854      0.525
Speed: 0.1ms preprocess, 5.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/runs/detect/volleyball_v2_finetune_all_aug
✅ Training complete!


In [4]:
from ultralytics import YOLO

# Load your trained model
model = YOLO('/kaggle/working/runs/detect/volleyball_v2_finetune_all_aug/weights/best.pt')

# Export it to ONNX format
model.export(format='onnx')

Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients

PyTorch: starting from '/kaggle/working/runs/detect/volleyball_v2_finetune_all_aug/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.11.13 environment at: /usr
Resolved 26 packages in 332ms
 Downloaded sympy
 Downloaded onnxruntime-gpu
Prepared 5 packages in 3.37s
Uninstalled 1 package in 215ms
Installed 5 packages in 53ms
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.74
 - sympy==1.13.1
 + sympy==1.14.0

requirements: AutoUpdate success ✅ 4.5s
WARNING ⚠️ requirements: Resta

'/kaggle/working/runs/detect/volleyball_v2_finetune_all_aug/weights/best.onnx'